# In this file we will evaluate the DVHs for the first three patients

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
fraction_dictionary = {
    '1a': 0,
    '1aA': 1,
    '1aB': 2,
    '1aC': 3,
    '1aD': 4,
    '1aE': 5,
}

## Code definition


In [2]:
def clean_dfs(dfs: list, overlaps:list, normalize_dose: bool = False):
    column_names =['Dosis', 'PTV', 'GTV', 'STOMACH', 'DUODENUM', 'BOWEL']
    PTV_volumes = []
    GTV_volumes = []
    STOMACH_volumes = []
    DUODENUM_volumes = []
    BOWEL_volumes = []
    overlap_percentage = []
    nested_lists = [PTV_volumes, GTV_volumes, STOMACH_volumes, DUODENUM_volumes, BOWEL_volumes]
    for index, df in enumerate(dfs):
        df.columns = df.columns.droplevel(0)
        for column in df.columns:
            for index_column, name in enumerate(column_names):
                if name in column:
                    df.rename(columns={column: name}, inplace=True)
                    if index_column!=0:
                        volume = float(column.split('Volumen:')[1].split('mL')[0].strip())
                        df[name] = df[name]*volume/100
                        if index%2 == 0:
                            nested_lists[index_column-1].append(volume)
                            if name == 'PTV':
                                overlap_percentage.append(overlaps[int(np.floor(index/2))]/volume*100)
        if normalize_dose:
            df['Dosis'] = df['Dosis']/5
        df.fillna(0, inplace=True)
        for column in df.columns[1:]:
            max_value = df[column].max()
            df[column + '%'] = df[column] / max_value * 100
    return dfs, overlap_percentage, PTV_volumes

## use steepness

In [3]:
import adaptive_fractionation_overlap as af
intercept = -0.660504
slope = -0.65

af.adaptfx_full(volumes = np.array([9.08,19.79,6.02,9.45,19.59,12.62]), number_of_fractions= 5, min_dose = 6, max_dose = 10, mean_dose = 6.6)

0 6.0
benefit 1.595430719999998
penalty 0.0
1 7.75
benefit 0
penalty 16.39599202
2 6.75
benefit 0
penalty 7.18721442
3 6.0
benefit 1.595430719999998
penalty 0.0
4 6.5
benefit 0.04431751999999968
penalty 6.31


(array([6.  , 7.75, 6.75, 6.  , 6.5 ]),
 array([ 0.  ,  6.  , 13.75, 20.5 , 26.5 ]),
 np.float64(-26.65802748))